In [85]:
using Revise
using Scattering
using LinearAlgebra
using BenchmarkTools
using Test
using Plots
using LaTeXStrings

In [86]:
Plots.default(size=(600, 370))
fntf = :Helvetica
titlefont = Plots.font(fntf, pointsize=12)
guidefont = Plots.font(fntf, pointsize=12)
tickfont = Plots.font(fntf, pointsize=9)
legendfont = Plots.font(fntf, pointsize=8)
Plots.default(fontfamily=fntf)
Plots.default(titlefont=titlefont, guidefont=guidefont, tickfont=tickfont, legendfont=legendfont)
Plots.default(minorticks=true)
Plots.default(linewidth=1.2)
Plots.default(foreground_color_legend=nothing)

## Testing form factor of a sphere

In [64]:
q1 = QVector([1, 0, 3.0])
q2 = QVector([1, 1, 2.0])
r = RVector([2.0, -3.0, 4.0])
r0 = [0.0, 0, 0]
qs = [q1, q2]

2-element Array{StaticArrays.SArray{Tuple{3},Float64,1,3},1}:
 [1.0, 0.0, 3.0]
 [1.0, 1.0, 2.0]

### Testing form factor for directional q

In [65]:
s0 = Sphere(2.0, 1.0, 1.0, r0)

Sphere{Float64}(2.0, 33.510321638291124, 1.0, 1.0, [0.0, 0.0, 0.0])

In [66]:
# test single q
formfactor(s0, q1)

-2.494688650014417 + 0.0im

In [69]:
formfactor_basic(s0, norm(q1))

-2.494688650014417

In [144]:
formfactor_basic(s0, 1.0)

56.76895855490903

In [70]:
@test formfactor(s0, q1) == formfactor_basic(s0, norm(q1))

Test Passed

In [71]:
@code_warntype formfactor(s0, q1)

Variables
  #self#::Core.Compiler.Const(Scattering.FormFactor.formfactor, false)
  s::Sphere{Float64}
  q::StaticArrays.SArray{Tuple{3},Float64,1,3}

Body::Complex{Float64}
1 ─ %1 = Scattering.FormFactor.composite(s)::Core.Compiler.Const(Basic(), false)
│   %2 = Scattering.FormFactor.formfactor(%1, s, q)::Complex{Float64}
└──      return %2


In [72]:
@code_warntype formfactor_basic(s0, norm(q1))

Variables
  #self#::Core.Compiler.Const(Scattering.FormFactor.formfactor_basic, false)
  s::Sphere{Float64}
  q::Float64
  qR::Float64

Body::Float64
1 ─ %1 = Base.getproperty(s, :R)::Float64
│        (qR = q * %1)
│   %3 = Base.getproperty(s, :Δρ)::Float64
│   %4 = Base.getproperty(s, :V)::Float64
│   %5 = Scattering.FormFactor.sphericalbesselj(1, qR)::Float64
│   %6 = (%3 * %4 * 3 * %5)::Float64
│   %7 = (%6 / qR)::Float64
└──      return %7


In [73]:
formfactor(s0, q2)

-1.6172529986543656 + 0.0im

In [74]:
# test a list of q
formfactors(s0, qs)

2-element Array{Complex{Float64},1}:
  -2.494688650014417 + 0.0im
 -1.6172529986543656 + 0.0im

In [75]:
@code_warntype formfactors(s0, qs)

Variables
  #self#::Core.Compiler.Const(Scattering.FormFactor.formfactors, false)
  s::Sphere{Float64}
  qs::Array{StaticArrays.SArray{Tuple{3},Float64,1,3},1}
  #65::Scattering.FormFactor.var"#65#66"{Sphere{Float64}}

Body::Array{Complex{Float64},1}
1 ─ %1 = Scattering.FormFactor.:(var"#65#66")::Core.Compiler.Const(Scattering.FormFactor.var"#65#66", false)
│   %2 = Core.typeof(s)::Core.Compiler.Const(Sphere{Float64}, false)
│   %3 = Core.apply_type(%1, %2)::Core.Compiler.Const(Scattering.FormFactor.var"#65#66"{Sphere{Float64}}, false)
│        (#65 = %new(%3, s))
│   %5 = #65::Scattering.FormFactor.var"#65#66"{Sphere{Float64}}
│   %6 = Base.Generator(%5, qs)::Base.Generator{Array{StaticArrays.SArray{Tuple{3},Float64,1,3},1},Scattering.FormFactor.var"#65#66"{Sphere{Float64}}}
│   %7 = Base.collect(%6)::Array{Complex{Float64},1}
└──      return %7


In [76]:
s = Sphere(2.0, 1.0, 1.0, r)

Sphere{Float64}(2.0, 33.510321638291124, 1.0, 1.0, [2.0, -3.0, 4.0])

In [77]:
formfactors(s, qs)

2-element Array{Complex{Float64},1}:
 -0.34111678629762715 + 2.4712569268727873im
  -1.2192506815289956 + 1.062513546853694im

In [78]:
@test formfactor(s0, q1) != formfactor(s, q1)

Test Passed

In [79]:
# test translating a sphere from origin to location r.
s1 = Scattering.translate(s0, r)
@test formfactors(s1, qs) ≈ formfactors(s, qs)

Test Passed

### Testing form factor for isotropic q

In [106]:
qxs = 0.01:0.01:10
qs = [QVector([qx, 0, 0]) for qx in qxs]
r0 = [0.0, 0, 0]
s0 = Sphere(1.0, 15.0, 15.0, r0)

Sphere{Float64}(1.0, 4.1887902047863905, 15.0, 15.0, [0.0, 0.0, 0.0])

In [145]:
formfactor_basic(s0, 1.0)

56.76895855490903

In [107]:
Fa = formfactors(s0, qs)
Pa = abs2.(Fa);

In [139]:
plot(qxs*s0.R, real(Fa)/(s0.V*s0.Δρ), xlabel=L"qR", ylabel=L"F(q)/(\Delta\rho V)", label=L"R=1.0, \Delta\rho=15.0")
savefig("/Users/lyx/Downloads/Fq_sphere.svg")

In [140]:
plot(qxs*s0.R, abs.(Fa)/(s0.V*s0.Δρ), yaxis=:log, xlabel=L"qR", ylabel=L"|F(q)|/(\Delta\rho V)", legend=false)
savefig("/Users/lyx/Downloads/Fq_sphere_ylog.svg")

In [141]:
plot(qxs*s0.R, Pa/(s0.V*s0.Δρ)^2, xlabel=L"qR", ylabel=L"I(q)/(\Delta\rho V)^2", label=L"R=1.0, \Delta\rho=15.0")
savefig("/Users/lyx/Downloads/Iq_sphere.svg")

In [143]:
plot(qxs*s0.R, Pa/(s0.V*s0.Δρ)^2, yaxis=:log, xlabel=L"qR", ylabel=L"I(q)/(\Delta\rho V)^2", legend=false)
# savefig("/Users/lyx/Downloads/Iq_sphere_ylog.svg")
savefig("/Users/lyx/Downloads/Iq_sphere_ylog.png")

In [131]:
r = RVector([2.0, -3.0, 4.0])
s1 = Scattering.translate(s0, r)
Fb = formfactors(s1, qs)
Pb = abs2.(Fb)
# squared form factor of a sphere should be invariant under translation.
@test Pa ≈ Pb

Test Passed